https://zerojsh00.github.io/posts/BERTopic/
https://towardsdatascience.com/topic-modeling-with-bert-779f7db187e6
https://arxiv.org/pdf/2203.05794.pdf

In [ ]:
! which python

In [ ]:
pip install bertopic

In [ ]:
pip uninstall umap


In [ ]:
pip install hdbscan

In [ ]:
pip install umap-learn

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pickle
import re
import umap.umap_ as umap
import hdbscan

In [2]:
# 파일 불러오기
with open('../../data/bert_df.pkl', 'rb') as f:
    df = pickle.load(f)


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3487 entries, 0 to 3486
Data columns (total 28 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   q_id                3487 non-null   int64         
 1   q_posttypeid        3487 non-null   object        
 2   q_acceptedanswerid  1488 non-null   float64       
 3   q_parentid          0 non-null      object        
 4   q_creationdate      3487 non-null   datetime64[ns]
 5   q_score             3487 non-null   int64         
 6   q_viewcount         3487 non-null   int64         
 7   q_owneruserid       3487 non-null   int64         
 8   q_title             3487 non-null   object        
 9   q_tags              3487 non-null   object        
 10  q_answercount       3487 non-null   int64         
 11  q_commentcount      3487 non-null   int64         
 12  q_reputation        3487 non-null   int64         
 13  q_text              3487 non-null   object      

In [4]:
# extract the  columns for text analysis
## q_body : question body
## a_body : answer body
df_qna = df[['q_id','q_text', 'a_text']]

In [5]:
df_qna.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3487 entries, 0 to 3486
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   q_id    3487 non-null   int64 
 1   q_text  3487 non-null   object
 2   a_text  3487 non-null   object
dtypes: int64(1), object(2)
memory usage: 81.9+ KB


In [6]:
def cleanhtml(raw_html):
  # 1.Source code in python language is hard to understand, so replace all the <code> tag first
  cleantext_1 = re.findall(r'(?<=\<code>)(.*?)(?=<\/code>)', raw_html.replace('\n', '_**_'))
  cleantext_1 = [x.replace('_**_', '\n') for x in cleantext_1]
  # 2. replace html tags
  # <p>
  tag_re = re.compile('<.*?>')
  cleantext_2 = [re.sub(tag_re, '', x) for x in cleantext_1]
  return cleantext_2

In [7]:
# apply the function, cleanhtml to the question and body text
df_qna.loc[:, 'q_prep_text'] = df_qna['q_text'].apply(cleanhtml)
df_qna.loc[:, 'a_prep_text'] = df_qna['a_text'].apply(cleanhtml)

/var/folders/lk/t243gzpx379bzd9mvx8dcsmm0000gp/T/ipykernel_67071/1671787946.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_qna.loc[:, 'q_prep_text'] = df_qna['q_text'].apply(cleanhtml)
/var/folders/lk/t243gzpx379bzd9mvx8dcsmm0000gp/T/ipykernel_67071/1671787946.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_qna.loc[:, 'a_prep_text'] = df_qna['a_text'].apply(cleanhtml)


In [8]:
df_qna.loc[:, [ 'q_id','q_prep_text', 'a_prep_text']] 

,q_id,q_prep_text,a_prep_text
0,77589765,"[M, N, n, M, M, N, M * N[None, None, :, None, ...","[In [29]: a,b,n,e,f = 2,3,4,5,6\nIn [30]: M=np..."
1,77591165,[import numpy as np\nimport pandas as pd\n\nim...,"[f = [client.submit(merge_noindex, d1, d2) \n ..."
2,77550047,[import pyautogui\npyautogui.displayMousePosit...,[]
3,77591174,"[list1=[0, 1, None, None, None, None, None, No...",[while len(fibonacci_sequence) &lt; n:\n fi...
4,77591245,"[__del__, __del__, cache = []\n\nclass Temp:\n...","[__del__, __del__, __del__, cache, atexit]"
...,...,...,...
3482,77591785,[indicadores_clean_pv = pd.melt(indicadores_cl...,"[.stack(), .unstack(), out = (\n df.set_ind..."
3483,77591847,"[4\n6\n7\n0\n, .XXX\n.X.X\n...X\nXXXX\n\n.XXXX...","[spiral_rec_southwest(M, xmin, xmax, ymin, yma..."
3484,77592073,[],[]
3485,77592121,[import random \n \n \nclass Pl...,"[select_hero(), import random \n ..."


In [9]:
df_q_src = df_qna[['q_id', 'q_prep_text']].apply(pd.Series.explode)
df_a_src = df_qna[['a_id', 'a_prep_text']].apply(pd.Series.explode)

In [45]:
df_q_src = df_q_src.reset_index(drop=True)
df_a_src = df_a_src.reset_index(drop=True)

In [46]:
df_q_src['q_prep_text'] = df_q_src['q_prep_text'].str.split('\n')
df_a_src['a_prep_text'] = df_a_src['a_prep_text'].str.split('\n')

KeyError: 'a_prep_text'

In [31]:
df_q_src.dropna(inplace=True)
df_a_src.dropna(inplace=True)

In [32]:
print(df_q_src['q_prep_text'].isna().sum())
print(df_a_src['a_prep_text'].isna().sum())

0

In [36]:
df_q_src['q_prep_text'] = df_q_src['q_prep_text'].apply(lambda x : [xx.lstrip().rstrip() for xx in x])
df_a_src['a_prep_text'] = df_a_src['a_prep_text'].apply(lambda x : [xx.lstrip().rstrip() for xx in x])

In [ ]:
# 전체 데이터셋 
with open('../../data/q_src_df.pkl', 'wb') as f:
    pickle.dump(df_q_src, f)

with open('../../data/a_src_df.pkl', 'wb') as f:
    pickle.dump(df_a_src, f)

In [11]:
# src_list = df_q_src.loc[df_q_src['q_id'] ==77592121, 'q_prep_text'].str.replace('\n', '').values[0]


'import random                  class Player:    def __init__(self, name, weapon, strength, health, attack, hero, creature):        self.name = name        self.weapon = weapon        self.strength = 100        self.health = 100        self.attack = 0        self.hero = hero        self.creature = creature            def entername_player(self):        self.name = input(&quot;Enter your name: &quot;)        print(&quot;Welcome &quot; + self.name + &quot; !&quot;)        return self.name                  def main():        hero = entername_player()                                    if __name__ == &quot;__main__&quot;:        main()    '

In [19]:
# src_list

'import random                  class Player:    def __init__(self, name, weapon, strength, health, attack, hero, creature):        self.name = name        self.weapon = weapon        self.strength = 100        self.health = 100        self.attack = 0        self.hero = hero        self.creature = creature            def entername_player(self):        self.name = input(&quot;Enter your name: &quot;)        print(&quot;Welcome &quot; + self.name + &quot; !&quot;)        return self.name                  def main():        hero = entername_player()                                    if __name__ == &quot;__main__&quot;:        main()    '

In [38]:
from transformers import AutoTokenizer, AutoModel
import torch

In [39]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
model = AutoModel.from_pretrained("microsoft/codebert-base")
nl_tokens=tokenizer.tokenize("return maximum value")

code_tokens=tokenizer.tokenize(src_list)

tokens=[tokenizer.cls_token]+nl_tokens+[tokenizer.sep_token]+code_tokens+[tokenizer.eos_token]

tokens_ids=tokenizer.convert_tokens_to_ids(tokens)

context_embeddings=model(torch.tensor(tokens_ids)[None,:])[0]


In [40]:
context_embeddings = context_embeddings.reshape(-1,768)

In [41]:
context_embeddings.shape

torch.Size([323, 768])

In [42]:

umap_embeddings = umap.UMAP(n_neighbors=15,
                            n_components=5,
                            metric='cosine').fit_transform(context_embeddings.detach().numpy())




huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [43]:
cluster = hdbscan.HDBSCAN(min_cluster_size=15,
                          metric='euclidean',
                          cluster_selection_method='eom').fit(umap_embeddings)

In [ ]:
cluster

In [ ]:
from transformers import RobertaConfig, RobertaTokenizer, RobertaForMaskedLM, pipeline

model = RobertaForMaskedLM.from_pretrained("microsoft/codebert-base-mlm")
tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base-mlm")

CODE = "if (x is not None) <mask> (x>1)"
fill_mask = pipeline('fill-mask', model=model, tokenizer=tokenizer)

outputs = fill_mask(CODE)
print(outputs)